Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [22]:
import logging
import numpy as np
import tensorflow as tf

from six.moves import range
from six.moves import cPickle as pickle

tf.get_logger().setLevel(logging.ERROR)


def reformat(dataset, labels, n_labels=10):
    dataset = dataset.reshape((-1, dataset.shape[1] * dataset.shape[-1])).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(n_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels


def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])


First reload the data we generated in `1_notmnist.ipynb`.

In [23]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [24]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Train a multinomial logistic regression using simple gradient descent.

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
n_input = 784
n_labels = 10

num_steps = 3001
display_step = 500
lr = 0.9

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
      tf.random.truncated_normal([n_input, n_labels]))
    biases = tf.Variable(tf.zeros([n_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

with tf.compat.v1.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.compat.v1.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % display_step == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.960957
Training accuracy: 8.2%
Validation accuracy: 15.5%
Loss at step 500: 1.041970
Training accuracy: 79.7%
Validation accuracy: 75.5%
Loss at step 1000: 0.759313
Training accuracy: 82.3%
Validation accuracy: 76.2%
Loss at step 1500: 0.630674
Training accuracy: 84.2%
Validation accuracy: 76.7%
Loss at step 2000: 0.555476
Training accuracy: 85.6%
Validation accuracy: 76.8%
Loss at step 2500: 0.505409
Training accuracy: 86.6%
Validation accuracy: 76.8%
Loss at step 3000: 0.469404
Training accuracy: 87.5%
Validation accuracy: 76.9%
Test accuracy: 84.2%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, n_input))
    tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, n_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.compat.v1.truncated_normal([n_input, n_labels]))
    biases = tf.Variable(tf.zeros([n_labels]))
  
    # Training computation.
    logits = tf.compat.v1.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.compat.v1.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.compat.v1.matmul(tf_test_dataset, weights) + biases)
    

with tf.compat.v1.Session(graph=graph) as session:
    tf.compat.v1.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % display_step == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.661783
Minibatch accuracy: 9.4%
Validation accuracy: 17.9%
Minibatch loss at step 500: 1.373304
Minibatch accuracy: 78.9%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 1.441033
Minibatch accuracy: 70.3%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 1.268058
Minibatch accuracy: 72.7%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 0.874608
Minibatch accuracy: 75.0%
Validation accuracy: 75.3%
Minibatch loss at step 2500: 1.116408
Minibatch accuracy: 74.2%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 1.071018
Minibatch accuracy: 72.7%
Validation accuracy: 79.0%
Test accuracy: 85.5%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [53]:
def multilayer_perceptron(x):
    layer_1 = tf.compat.v1.matmul(x, weights['h1']) + biases['b1']
    activation = tf.nn.relu(layer_1)
    output = tf.compat.v1.matmul(activation, weights['out']) + biases['out']
    
    return output


graph = tf.Graph()
with graph.as_default():
    # Graph input
    tf_train_dataset = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, n_input))
    tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, n_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Layers parameters
    n_hidden=1024
    weights = {
        'h1': tf.Variable(tf.compat.v1.truncated_normal([n_input, n_hidden])),
        'out': tf.Variable(tf.compat.v1.truncated_normal([n_hidden, n_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden])), 'out': tf.Variable(tf.zeros([n_labels]))
    }
  
    # Training computation.
    logits = multilayer_perceptron(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset))
    test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset))
    

with tf.compat.v1.Session(graph=graph) as session:
    tf.compat.v1.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 340.662781
Minibatch accuracy: 14.1%
Validation accuracy: 11.8%
Minibatch loss at step 1: 3368.171387
Minibatch accuracy: 7.8%
Validation accuracy: 37.3%
Minibatch loss at step 2: 4051.207764
Minibatch accuracy: 35.2%
Validation accuracy: 44.8%
Minibatch loss at step 3: 1510.760986
Minibatch accuracy: 50.8%
Validation accuracy: 39.9%
Minibatch loss at step 4: 1240.955933
Minibatch accuracy: 32.0%
Validation accuracy: 46.4%
Minibatch loss at step 5: 535.945374
Minibatch accuracy: 48.4%
Validation accuracy: 52.6%
Minibatch loss at step 6: 815.183350
Minibatch accuracy: 45.3%
Validation accuracy: 55.8%
Minibatch loss at step 7: 351.992126
Minibatch accuracy: 57.0%
Validation accuracy: 57.3%
Minibatch loss at step 8: 470.066315
Minibatch accuracy: 54.7%
Validation accuracy: 58.9%
Minibatch loss at step 9: 310.880371
Minibatch accuracy: 60.2%
Validation accuracy: 65.8%
Minibatch loss at step 10: 222.352600
Minibatch accuracy: 71.1%
Validation accuracy: 

Validation accuracy: 71.5%
Minibatch loss at step 107: 58.866383
Minibatch accuracy: 64.1%
Validation accuracy: 70.1%
Minibatch loss at step 108: 193.224167
Minibatch accuracy: 62.5%
Validation accuracy: 70.4%
Minibatch loss at step 109: 25.989674
Minibatch accuracy: 72.7%
Validation accuracy: 72.3%
Minibatch loss at step 110: 64.984077
Minibatch accuracy: 75.0%
Validation accuracy: 70.5%
Minibatch loss at step 111: 61.030159
Minibatch accuracy: 71.1%
Validation accuracy: 72.7%
Minibatch loss at step 112: 44.240894
Minibatch accuracy: 73.4%
Validation accuracy: 73.4%
Minibatch loss at step 113: 47.406803
Minibatch accuracy: 74.2%
Validation accuracy: 73.9%
Minibatch loss at step 114: 30.448210
Minibatch accuracy: 77.3%
Validation accuracy: 74.5%
Minibatch loss at step 115: 41.854309
Minibatch accuracy: 72.7%
Validation accuracy: 74.9%
Minibatch loss at step 116: 88.314491
Minibatch accuracy: 74.2%
Validation accuracy: 72.3%
Minibatch loss at step 117: 37.757538
Minibatch accuracy: 76.6

Validation accuracy: 73.0%
Minibatch loss at step 213: 34.299911
Minibatch accuracy: 71.9%
Validation accuracy: 76.4%
Minibatch loss at step 214: 12.696467
Minibatch accuracy: 77.3%
Validation accuracy: 75.6%
Minibatch loss at step 215: 10.072228
Minibatch accuracy: 75.8%
Validation accuracy: 75.5%
Minibatch loss at step 216: 25.713192
Minibatch accuracy: 72.7%
Validation accuracy: 75.6%
Minibatch loss at step 217: 33.525707
Minibatch accuracy: 75.8%
Validation accuracy: 74.5%
Minibatch loss at step 218: 26.185764
Minibatch accuracy: 74.2%
Validation accuracy: 75.7%
Minibatch loss at step 219: 24.250118
Minibatch accuracy: 75.8%
Validation accuracy: 75.6%
Minibatch loss at step 220: 13.418213
Minibatch accuracy: 75.0%
Validation accuracy: 74.8%
Minibatch loss at step 221: 45.780876
Minibatch accuracy: 73.4%
Validation accuracy: 74.0%
Minibatch loss at step 222: 14.421941
Minibatch accuracy: 77.3%
Validation accuracy: 74.0%
Minibatch loss at step 223: 28.036922
Minibatch accuracy: 75.0%

Validation accuracy: 74.8%
Minibatch loss at step 305: 29.025259
Minibatch accuracy: 75.8%
Validation accuracy: 73.1%
Minibatch loss at step 306: 47.058899
Minibatch accuracy: 72.7%
Validation accuracy: 70.4%
Minibatch loss at step 307: 25.267292
Minibatch accuracy: 70.3%
Validation accuracy: 76.0%
Minibatch loss at step 308: 17.245651
Minibatch accuracy: 72.7%
Validation accuracy: 77.2%
Minibatch loss at step 309: 35.348709
Minibatch accuracy: 76.6%
Validation accuracy: 74.7%
Minibatch loss at step 310: 22.540779
Minibatch accuracy: 78.1%
Validation accuracy: 75.4%
Minibatch loss at step 311: 34.549835
Minibatch accuracy: 71.1%
Validation accuracy: 73.4%
Minibatch loss at step 312: 33.216461
Minibatch accuracy: 82.0%
Validation accuracy: 75.5%
Minibatch loss at step 313: 22.821278
Minibatch accuracy: 79.7%
Validation accuracy: 74.0%
Minibatch loss at step 314: 52.044983
Minibatch accuracy: 76.6%
Validation accuracy: 72.1%
Minibatch loss at step 315: 24.289053
Minibatch accuracy: 77.3%

Minibatch loss at step 416: 14.567692
Minibatch accuracy: 69.5%
Validation accuracy: 76.1%
Minibatch loss at step 417: 17.229939
Minibatch accuracy: 77.3%
Validation accuracy: 76.7%
Minibatch loss at step 418: 7.574613
Minibatch accuracy: 78.9%
Validation accuracy: 77.0%
Minibatch loss at step 419: 18.067226
Minibatch accuracy: 77.3%
Validation accuracy: 76.7%
Minibatch loss at step 420: 8.619024
Minibatch accuracy: 79.7%
Validation accuracy: 77.6%
Minibatch loss at step 421: 25.104612
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 422: 14.497746
Minibatch accuracy: 71.9%
Validation accuracy: 77.8%
Minibatch loss at step 423: 25.568329
Minibatch accuracy: 71.9%
Validation accuracy: 77.0%
Minibatch loss at step 424: 39.453594
Minibatch accuracy: 68.0%
Validation accuracy: 74.8%
Minibatch loss at step 425: 24.587120
Minibatch accuracy: 75.0%
Validation accuracy: 77.1%
Minibatch loss at step 426: 10.110834
Minibatch accuracy: 79.7%
Validation accuracy: 76.0%
M

...

Validation accuracy: 80.4%
Minibatch loss at step 2533: 4.389533
Minibatch accuracy: 82.0%
Validation accuracy: 80.8%
Minibatch loss at step 2534: 4.326018
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Minibatch loss at step 2535: 1.342140
Minibatch accuracy: 85.2%
Validation accuracy: 81.0%
Minibatch loss at step 2536: 1.996692
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2537: 2.535675
Minibatch accuracy: 83.6%
Validation accuracy: 80.4%
Minibatch loss at step 2538: 1.304432
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 2539: 2.208982
Minibatch accuracy: 85.2%
Validation accuracy: 81.0%
Minibatch loss at step 2540: 1.528246
Minibatch accuracy: 82.0%
Validation accuracy: 80.4%
Minibatch loss at step 2541: 2.290766
Minibatch accuracy: 82.8%
Validation accuracy: 81.3%
Minibatch loss at step 2542: 0.787869
Minibatch accuracy: 86.7%
Validation accuracy: 81.5%
Minibatch loss at step 2543: 4.426960
Minibatch accuracy: 77.3%

Validation accuracy: 81.1%
Minibatch loss at step 2644: 1.686584
Minibatch accuracy: 84.4%
Validation accuracy: 81.4%
Minibatch loss at step 2645: 2.246660
Minibatch accuracy: 80.5%
Validation accuracy: 81.1%
Minibatch loss at step 2646: 1.721025
Minibatch accuracy: 86.7%
Validation accuracy: 80.7%
Minibatch loss at step 2647: 2.113252
Minibatch accuracy: 82.0%
Validation accuracy: 80.5%
Minibatch loss at step 2648: 4.236852
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 2649: 2.020432
Minibatch accuracy: 85.9%
Validation accuracy: 81.2%
Minibatch loss at step 2650: 3.786527
Minibatch accuracy: 80.5%
Validation accuracy: 79.0%
Minibatch loss at step 2651: 3.779359
Minibatch accuracy: 76.6%
Validation accuracy: 77.6%
Minibatch loss at step 2652: 2.969997
Minibatch accuracy: 79.7%
Validation accuracy: 80.5%
Minibatch loss at step 2653: 2.385463
Minibatch accuracy: 83.6%
Validation accuracy: 80.4%
Minibatch loss at step 2654: 3.351329
Minibatch accuracy: 83.6%

Minibatch loss at step 2752: 2.667992
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 2753: 1.882058
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 2754: 1.037238
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 2755: 2.006171
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 2756: 3.072750
Minibatch accuracy: 73.4%
Validation accuracy: 80.2%
Minibatch loss at step 2757: 2.889851
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 2758: 2.649766
Minibatch accuracy: 85.2%
Validation accuracy: 80.5%
Minibatch loss at step 2759: 1.870762
Minibatch accuracy: 82.0%
Validation accuracy: 80.5%
Minibatch loss at step 2760: 0.581102
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 2761: 1.740650
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Minibatch loss at step 2762: 1.624674
Minibatch accuracy: 89.1%
Validation accuracy: 81.3%

Validation accuracy: 81.0%
Minibatch loss at step 2862: 1.504397
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2863: 1.813000
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 2864: 2.588033
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 2865: 2.450684
Minibatch accuracy: 79.7%
Validation accuracy: 81.4%
Minibatch loss at step 2866: 2.820218
Minibatch accuracy: 77.3%
Validation accuracy: 80.9%
Minibatch loss at step 2867: 2.352181
Minibatch accuracy: 86.7%
Validation accuracy: 81.1%
Minibatch loss at step 2868: 2.250105
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 2869: 2.970646
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 2870: 1.420079
Minibatch accuracy: 86.7%
Validation accuracy: 81.1%
Minibatch loss at step 2871: 2.253541
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2872: 2.705873
Minibatch accuracy: 80.5%

Validation accuracy: 80.6%
Minibatch loss at step 2953: 1.255003
Minibatch accuracy: 85.2%
Validation accuracy: 80.7%
Minibatch loss at step 2954: 1.429459
Minibatch accuracy: 75.0%
Validation accuracy: 81.1%
Minibatch loss at step 2955: 3.903102
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 2956: 2.020429
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2957: 1.940093
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 2958: 3.009456
Minibatch accuracy: 80.5%
Validation accuracy: 80.9%
Minibatch loss at step 2959: 1.366941
Minibatch accuracy: 79.7%
Validation accuracy: 80.8%
Minibatch loss at step 2960: 2.715522
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 2961: 2.946275
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 2962: 1.929261
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 2963: 0.765754
Minibatch accuracy: 82.8%